## Librerias

import pandas as pd
from pathlib import Path
import numpy as np
import os
import datetime as dt
import warnings
import re, unicodedata


In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
import datetime as dt
import warnings
import re, unicodedata
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [2]:
# Fijar directorio
DATA_DIR = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\raw")
DATA_OUT = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed")

In [3]:
# Lista todos los archivos .csv
csv_files = sorted(DATA_DIR.glob("*.csv"))
print(f"Se listan {len(csv_files)} archivos a procesar:")
for f in csv_files:
    print("-", f.name)

## Lectura de datos
data_raw = []
cols_by_file = {}
for file in csv_files:
    df = pd.read_csv(file, sep="|")
    df["file_origin"] = file.name
    data_raw.append(df)
    cols_by_file[file.name] = set(df.columns)
    
print("\nproceso cargado!")

print("\nUniendo datos!")
raw_all = pd.concat(data_raw, ignore_index=True)
print("Filas totales:", len(raw_all))
print("Columnas:", list(raw_all.columns))

Se listan 35 archivos a procesar:
- data_alquiler_departamentos_adondevivir_Lima Centro.csv
- data_alquiler_departamentos_adondevivir_Lima Este.csv
- data_alquiler_departamentos_adondevivir_Lima Moderna.csv
- data_alquiler_departamentos_adondevivir_Lima Norte.csv
- data_alquiler_departamentos_adondevivir_Lima Sur.csv
- data_alquiler_departamentos_adondevivir_Lima Top.csv
- data_alquiler_departamentos_urbania_Lima Centro.csv
- data_alquiler_departamentos_urbania_Lima Este.csv
- data_alquiler_departamentos_urbania_Lima Moderna.csv
- data_alquiler_departamentos_urbania_Lima Norte.csv
- data_alquiler_departamentos_urbania_Lima Sur.csv
- data_alquiler_departamentos_urbania_Lima Top.csv
- data_venta_casas_adondevivir_Lima Centro.csv
- data_venta_casas_adondevivir_Lima Este.csv
- data_venta_casas_adondevivir_Lima Moderna.csv
- data_venta_casas_adondevivir_Lima Norte.csv
- data_venta_casas_adondevivir_Lima Sur.csv
- data_venta_casas_adondevivir_Lima Top.csv
- data_venta_casas_urbania_Lima Cent

In [4]:
print("\nLimpieza de datos!")

## Limpieza de datos - duplicados Otros. 

dups = raw_all["enlace"].duplicated(keep=False)
print("Registros duplicados por enlace:", dups.sum())

# ¿Cuántos enlaces únicos?
print("Enlaces únicos:", raw_all["enlace"].nunique())

## removeremos duplicados.
raw_all_nd = raw_all.drop_duplicates(subset=["enlace"], keep="first") 
print("se tiene solo enlaces unicos:", raw_all_nd["enlace"].nunique())


Limpieza de datos!
Registros duplicados por enlace: 159
Enlaces únicos: 49767
se tiene solo enlaces unicos: 49767


In [5]:
valores = {
    "Brena": "Breña",
    "Jesus Maria": "Jesús María",
    "San Martin De Porres": "San Martín de Porres",
    "Lurin": "Lurín",
    "Ancon": "Ancón",
    "Santa Maria Del Mar": "Santa María del Mar",
    "Villa Maria Del Triunfo": "Villa María del Triunfo",
}

s = (raw_all_nd["distrito"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True))

raw_all_nd["distrito"] = s.replace(valores)   # crea la nueva columna

raw_all_nd["distrito_norm"] = raw_all_nd["distrito"].str.lower().str.strip()
raw_all_nd["direccion_norm"] = raw_all_nd["direccion"].str.lower().str.strip()

raw_all_nd["distrito_en_direccion"] = raw_all_nd.apply(
    lambda row: row["distrito_norm"] in row["direccion_norm"],
    axis=1
)

C:\Users\PC\AppData\Local\Temp\ipykernel_25784\3101019241.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["distrito"] = s.replace(valores)   # crea la nueva columna
C:\Users\PC\AppData\Local\Temp\ipykernel_25784\3101019241.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["distrito_norm"] = raw_all_nd["distrito"].str.lower().str.strip()
C:\Users\PC\AppData\Local\Temp\ipykernel_25784\3101019241.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [6]:
cols = ["distrito", "direccion", 
        "distrito_norm", "direccion_norm", "distrito_en_direccion"]
raw_all_nd[raw_all_nd["distrito_en_direccion"]==False][cols]

,distrito,direccion,distrito_norm,direccion_norm,distrito_en_direccion
60,Breña,"Mariano Carranza al 400\nSanta Beatriz, Lima C...",breña,"mariano carranza al 400\nsanta beatriz, lima c...",False
61,Breña,"faustino sanchez carrion 397\nJesús María, Lima",breña,"faustino sanchez carrion 397\njesús maría, lima",False
62,Breña,"Jirón Estados Unidos 801\nJesús María, Lima",breña,"jirón estados unidos 801\njesús maría, lima",False
63,Breña,"Jirón Diego de Almagro\nJesús María, Lima",breña,"jirón diego de almagro\njesús maría, lima",False
64,Breña,"Calle Arnaldo Marquez 1440\nJesús María, Lima",breña,"calle arnaldo marquez 1440\njesús maría, lima",False
...,...,...,...,...,...
39209,Villa Maria del Triunfo,Jr. José Olaya Mz. D6 Lt. 12\nSanta Rosa De Be...,villa maria del triunfo,jr. josé olaya mz. d6 lt. 12\nsanta rosa de be...,False
39210,Villa Maria del Triunfo,Villa María Del Triunfo\nVilla María del Triun...,villa maria del triunfo,villa maría del triunfo\nvilla maría del triun...,False
39211,Villa Maria del Triunfo,"Castilla 245\nVilla María del Triunfo, Lima",villa maria del triunfo,"castilla 245\nvilla maría del triunfo, lima",False
39212,Villa Maria del Triunfo,"Castilla 245\nVilla María del Triunfo, Lima",villa maria del triunfo,"castilla 245\nvilla maría del triunfo, lima",False


In [7]:
canon = {
    # canónico : [alias que podrían aparecer en la dirección]
    "Ate Vitarte": ["ate vitarte", "ate"],
    "Breña": ["breña", "brena"],
    "Carabayllo": ["carabayllo"],
    "Chaclacayo": ["chaclacayo"],
    "El Agustino": ["el agustino", "agustino"],
    "Independencia": ["independencia"],
    "Jesús María": ["jesus maria", "jesús maria", "jesus maría"], ## Jesús María
    "Chosica Lurigancho": [ "chosica"],
    "Lurín": ["lurin", "lurín"],
    "Pachacamac": ["pachacamac", "pachacámac"],
    "Pucusana": ["pucusana"],
    "Puente Piedra": ["puente piedra"],
    "Punta Negra": ["punta negra"],
    "Rimac": ["rimac", "rímac"],
    "San Juan de Miraflores": ["san juan de miraflores", "sjm"],
    "San Luis": ["san luis"],
    "San Martin de Porres": ["san martin de porres", "san martín de porres", "smp"],
    "Villa Maria del Triunfo": ["villa maria del triunfo", "villa maría del triunfo", "vmt"],
    "Ancón": ["ancon", "ancón"],
    "Lince": ["lince"],
    "Santa Rosa": ["santa rosa"],
    "Barranco": ["barranco"],
    "Magdalena": ["magdalena del mar", "magdalena"],
    "Punta Hermosa": ["punta hermosa"],
    "Santiago de Surco": ["santiago de surco", "surco"],
    "San Juan de Lurigancho": ["san juan de lurigancho"],
    "Lima Cercado" : ["cercado", "cercado de lima", "lima cercado"],
    "Los Olivos" : ["los olivos"],
    "Comas" : ["comas"],
    "San Bartolo" : ["san bartolo"], 
    'Villa El Salvador' : ["villa el salvador"],
    "Santa Maria del Mar": ["santa maria del mar"],
    "San Isidro" : ["san isidro"],
    "La Molina" : ["la molina"],
    "Chorrillos" : ["chorrillos"],
    "San Borja" : ["san borja"],
    "Miraflores" : ["miraflores"],
    "Surquillo" : ["surquillo"],
    "Cieneguilla" : ["cieneguilla"],
    "Ventanilla" : ["ventanilla"],
    "La Victoria" : ["la victoria"],
    'Pueblo Libre' : ["pueblo libre"], 
    'San Miguel' : ["san miguel"], 
    'Santa Anita' : ["santa anita"], 
    'Ventanilla' : ["ventanilla"], 
}

def strip_accents(s):
    if not isinstance(s, str):
        return s
    return ''.join(ch for ch in unicodedata.normalize("NFD", s) if unicodedata.category(ch) != "Mn")

raw_all_nd["dir_norm"]  = raw_all_nd["direccion"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
raw_all_nd["dist_norm"] = raw_all_nd["distrito"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)

# precompila patrones regex alias
patterns = []
for canon_name, aliases in canon.items():
    for alias in aliases:
        pat = re.compile(rf"\b{re.escape(strip_accents(alias))}\b")
        patterns.append((canon_name, pat))

def candidates_from_address(addr_norm: str):
    if not isinstance(addr_norm, str):
        return []
    hits = []
    for canon_name, pat in patterns:
        if pat.search(addr_norm):
            hits.append(canon_name)
    # quitar duplicados preservando orden
    return  list(dict.fromkeys(hits))

raw_all_nd["dist_candidates"] = raw_all_nd["dir_norm"].map(candidates_from_address)
raw_all_nd["distrito_candidato"] = raw_all_nd["dist_candidates"].str.get(0) 

C:\Users\PC\AppData\Local\Temp\ipykernel_25784\2080104380.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["dir_norm"]  = raw_all_nd["direccion"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
C:\Users\PC\AppData\Local\Temp\ipykernel_25784\2080104380.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["dist_norm"] = raw_all_nd["distrito"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
C:\Users\PC\AppData\Local\Temp\

In [8]:
raw_all_nd[(raw_all_nd["distrito_en_direccion"]==False) & (raw_all_nd["distrito_candidato"]).isnull()][cols + ["distrito_candidato", "enlace", "operacion"]]

,distrito,direccion,distrito_norm,direccion_norm,distrito_en_direccion,distrito_candidato,enlace,operacion
170,San Luis,Malecon Rivera DEL MAR 1 DPT 4XX B\nCerro Azul...,san luis,malecon rivera del mar 1 dpt 4xx b\ncerro azul...,False,NaN,https://www.adondevivir.com/propiedades/clasif...,alquiler
172,San Luis,"Playa Puerto Fiel, Cerro Azul, Cañete\nCerro A...",san luis,"playa puerto fiel, cerro azul, cañete\ncerro a...",False,NaN,https://www.adondevivir.com/propiedades/clasif...,alquiler
173,San Luis,"SAN Antonio DE Mala\nSan Vicente De Cañete, Ca...",san luis,"san antonio de mala\nsan vicente de cañete, ca...",False,NaN,https://www.adondevivir.com/propiedades/clasif...,alquiler
7830,San Luis,"Cerro colorado\nCerro Azul, Cerro Azul",san luis,"cerro colorado\ncerro azul, cerro azul",False,NaN,https://www.adondevivir.com/propiedades/clasif...,venta
7831,San Luis,Condominio Kala - Puerto Viejo\nSan Vicente De...,san luis,condominio kala - puerto viejo\nsan vicente de...,False,NaN,https://www.adondevivir.com/propiedades/clasif...,venta
7832,San Luis,"Condominio Playa Azul\nCerro Azul, Cerro Azul",san luis,"condominio playa azul\ncerro azul, cerro azul",False,NaN,https://www.adondevivir.com/propiedades/clasif...,venta
7833,San Luis,"San Vicente de Cañete\nSan Vicente De Cañete, ...",san luis,"san vicente de cañete\nsan vicente de cañete, ...",False,NaN,https://www.adondevivir.com/propiedades/clasif...,venta
7834,San Luis,"Playa Punta Corrientes\nCerro Azul, Cañete",san luis,"playa punta corrientes\ncerro azul, cañete",False,NaN,https://www.adondevivir.com/propiedades/clasif...,venta
7835,San Luis,"Casa en Totoritas, Mala\nSan Vicente De Cañete...",san luis,"casa en totoritas, mala\nsan vicente de cañete...",False,NaN,https://www.adondevivir.com/propiedades/clasif...,venta
7836,San Luis,"Playa Lomas del Mar\nLomas De Mar, Cerro Azul",san luis,"playa lomas del mar\nlomas de mar, cerro azul",False,NaN,https://www.adondevivir.com/propiedades/clasif...,venta


In [11]:
raw_all_nd2 = raw_all_nd[(raw_all_nd["distrito_candidato"].notnull()) & (raw_all_nd["distrito_candidato"] != "Ventanilla") ].copy() ## no van distritos de callao ni distritos candidatos qu eno pertenezcan a lima

In [12]:
raw_all_nd2.head()

,fuente,inmueble,operacion,zona,distrito,direccion,precio,soles,dolares,variacion,...,enlace,fecha,file_origin,distrito_norm,direccion_norm,distrito_en_direccion,dir_norm,dist_norm,dist_candidates,distrito_candidato
0,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,lima cercado,"mariano carranza al 400\nsanta beatriz, lima c...",True,"mariano carranza al 400 santa beatriz, lima ce...",lima cercado,[Lima Cercado],Lima Cercado
1,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,lima cercado,"avenida colonial 3046\nla colonial, lima cercado",True,"avenida colonial 3046 la colonial, lima cercado",lima cercado,[Lima Cercado],Lima Cercado
2,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,lima cercado,"parque de la reserva 736\nlima cercado, lima",True,"parque de la reserva 736 lima cercado, lima",lima cercado,[Lima Cercado],Lima Cercado
3,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,lima cercado,"av. alejandro bertello cdra\npando, lima cercado",True,"av. alejandro bertello cdra pando, lima cercado",lima cercado,[Lima Cercado],Lima Cercado
4,adondevivir,departamentos,alquiler,Lima Centro,Lima Cercado,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,lima cercado,"av. oscar r. benavides\nlima cercado, lima",True,"av. oscar r. benavides lima cercado, lima",lima cercado,[Lima Cercado],Lima Cercado


In [13]:

data_p1 = raw_all_nd

cols_obj =['fuente', 'inmueble', 'operacion', 'distrito_candidato' , 'zona', 'direccion', 'precio',
            'soles', 'dolares', 'variacion', 'mantenimiento', 'caracteristica',
            'area', 'dormitorio', 'baños', 'estacionamientos', 'detalle', 'enlace',
            'fecha', 'file_origin']


data_vivienda = raw_all_nd2[cols_obj].copy()

def normalizar(texto: str) -> str:
    # 1. Pasar a minúsculas
    texto = texto.lower()
    # 2. Quitar acentos
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )
    # 3. Quitar dobles espacios y limpiar
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto


data_vivienda["distrito_oficial"] = data_vivienda["distrito_candidato"] ## .apply(normalizar)

In [14]:
data_vivienda.head()

,fuente,inmueble,operacion,distrito_candidato,zona,direccion,precio,soles,dolares,variacion,...,caracteristica,area,dormitorio,baños,estacionamientos,detalle,enlace,fecha,file_origin,distrito_oficial
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,61 m² tot.\n2 dorm.\n2 baños,61,2,2.0,NaN,Departamento de estreno con Balcón y Excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,60 m² tot.\n3 dorm.\n2 baños,60,3,2.0,NaN,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,75 m² tot.\n3 dorm.\n2 baños,75,3,2.0,NaN,Contacto: malena peláez // cel: 993*306*450. S...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,32 m² tot.\n1 dorm.\n1 baño,32,1,NaN,NaN,Alquiler de Departamento en Lima Cercado - Urb...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,50 m² tot.\n2 dorm.\n2 baños,50,2,2.0,NaN,"Con un área de 50m2, Ubicado en el Piso 14 con...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado


In [15]:
data_vivienda[data_vivienda["distrito_candidato"]=="Independencia"].head(50)

,fuente,inmueble,operacion,distrito_candidato,zona,direccion,precio,soles,dolares,variacion,...,caracteristica,area,dormitorio,baños,estacionamientos,detalle,enlace,fecha,file_origin,distrito_oficial
155,adondevivir,departamentos,alquiler,Independencia,Lima Centro,"MegaPlaza\nPanamericana Norte Industrial, Inde...","S/ 1,300 · USD 380",1300.0,380.0,NaN,...,85 m² tot.\n3 dorm.\n2 baños,85,3,2.0,NaN,"Departamento en piso 3, cuenta con 3 habitacio...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Independencia
319,adondevivir,departamentos,alquiler,Independencia,Lima Este,"MegaPlaza\nPanamericana Norte Industrial, Inde...","S/ 1,300 · USD 380",1300.0,380.0,NaN,...,85 m² tot.\n3 dorm.\n2 baños,85,3.0,2.0,NaN,"Departamento en piso 3, cuenta con 3 habitacio...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima E...,Independencia
684,adondevivir,departamentos,alquiler,Independencia,Lima Moderna,"Jr. Independencia 508 San Miguel\nSan Miguel, ...",S/ 880,880.0,NaN,NaN,...,25 m² tot.\n1 dorm.\n1 baño,25,1.0,NaN,NaN,Alquilo ambiente amplio tipo estudio en 5to. p...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,data_alquiler_departamentos_adondevivir_Lima M...,Independencia
767,adondevivir,departamentos,alquiler,Independencia,Lima Moderna,"jr. Independencia\nSan Miguel, Lima",S/ 900 · USD 240,900.0,240.0,NaN,...,28 m² tot.\n1 dorm.\n1 baño,28,1.0,NaN,NaN,Contáctame Susan Alzamora. 9. 0. 2. 0. 1. 6. 4...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,data_alquiler_departamentos_adondevivir_Lima M...,Independencia
1133,adondevivir,departamentos,alquiler,Independencia,Lima Norte,"MegaPlaza\nPanamericana Norte Industrial, Inde...","S/ 1,300 · USD 380",1300.0,380.0,NaN,...,85 m² tot.\n3 dorm.\n2 baños,85.0,3.0,2.0,NaN,"Departamento en piso 3, cuenta con 3 habitacio...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima N...,Independencia
1134,adondevivir,departamentos,alquiler,Independencia,Lima Norte,"Los Melocotones\nIndependencia, Lima",S/ 900 · USD 257,900.0,257.0,NaN,...,77 m² tot.\n2 dorm.\n1 baño,77.0,2.0,NaN,NaN,"Se alquila departamento en 2do piso, de 78m² u...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima N...,Independencia
1135,adondevivir,departamentos,alquiler,Independencia,Lima Norte,"Los Capulies 121 Independencia\nLas Violetas, ...","S/ 1,800",1800.0,NaN,NaN,...,140 m² tot.\n3 dorm.\n2 baños,140.0,3.0,2.0,NaN,Departamento muy amplio semi amoblado. Todo el...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima N...,Independencia
1136,adondevivir,departamentos,alquiler,Independencia,Lima Norte,"LAS AMERICAS\nIndependencia, Lima",S/ 899,899.0,NaN,NaN,...,60 m² tot.\n2 dorm.\n1 baño,60.0,2.0,NaN,NaN,*Alquiler de departamento en Independencia! * ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima N...,Independencia
1655,adondevivir,departamentos,alquiler,Independencia,Lima Top,"Independencia\nMiraflores, Lima","S/ 3,000",3000.0,NaN,NaN,...,93 m² tot.\n2 dorm.\n2 baños\n1 estac.,93,2,2.0,1.0,Alquiler mensual s/3. 000 Precioso departament...,https://www.adondevivir.com/propiedades/clasif...,2025-12-31,data_alquiler_departamentos_adondevivir_Lima T...,Independencia
1670,adondevivir,departamentos,alquiler,Independencia,Lima Top,"Calle Independencia 4, Piso 4, Miraflores\nMir...","S/ 2,958 · USD 870",2958.0,870.0,NaN,...,80 m² tot.\n2 dorm.\n1 baño\n1 estac.,80,2,NaN,1.0,"Alquilo Lindo Departamento Flat Remodelado, Fu...",https://www.adondevivir.com/propiedades/clasif...,2025-12-31,data_alquiler_departamentos_adondevivir_Lima T...,Independencia


In [16]:

print("\nCreacion de variables!")

cols_analisis = ['fuente','inmueble','operacion',
                'distrito_oficial', 'zona',
                'direccion','precio',
                'soles','dolares','variacion','mantenimiento','caracteristica',
                'area','dormitorio','baños','estacionamientos','detalle','enlace',
                'fecha','file_origin']
data_analisis = data_vivienda[cols_analisis].copy()
data_analisis


Creacion de variables!


,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,mantenimiento,caracteristica,area,dormitorio,baños,estacionamientos,detalle,enlace,fecha,file_origin
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,NaN,61 m² tot.\n2 dorm.\n2 baños,61,2,2.0,NaN,Departamento de estreno con Balcón y Excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,160.0,60 m² tot.\n3 dorm.\n2 baños,60,3,2.0,NaN,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,NaN,75 m² tot.\n3 dorm.\n2 baños,75,3,2.0,NaN,Contacto: malena peláez // cel: 993*306*450. S...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,180.0,32 m² tot.\n1 dorm.\n1 baño,32,1,NaN,NaN,Alquiler de Departamento en Lima Cercado - Urb...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,210.0,50 m² tot.\n2 dorm.\n2 baños,50,2,2.0,NaN,"Con un área de 50m2, Ubicado en el Piso 14 con...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49846,urbania,departamentos,venta,Barranco,Lima Top,"Pasaje Bresciani 1\nBarranco, Lima","S/ 6,146,200 · USD 1,580,000",6146200.0,1580000.0,NaN,"4,000",638 m² tot.\n3 dorm.\n3 baños\n7 estac.,638,3,3.0,7.0,"Vendo Lindo departamento frente al Mar, en Pas...",https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv
49847,urbania,departamentos,venta,Barranco,Lima Top,"Jirón Tumay 095\nBarranco, Lima","S/ 603,000 · USD 167,500",603000.0,167500.0,NaN,400,62 m² tot.\n2 dorm.\n2 baños,62,2,2.0,NaN,Departamento en venta en BarrancoExcelente ubi...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv
49848,urbania,departamentos,venta,Barranco,Lima Top,"martinto 150\nBarranco, Lima","S/ 562,800 · USD 168,000",562800.0,168000.0,NaN,200,86 m² tot.\n2 dorm.\n2 baños\n2 estac.,86,2,2.0,2.0,"Lindo dpto. Muy bien distribuido, de 2 Dormito...",https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv
49849,urbania,departamentos,venta,Barranco,Lima Top,"Ignacio mariategui\nBarranco, Lima","S/ 351,750 · USD 105,000",351750.0,105000.0,NaN,180,76 m² tot.\n2 dorm.\n1 baño,76,2,NaN,NaN,Hermoso departamento en venta en Barranco en p...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv


In [17]:
# Normalizaciones básicas
data_analisis["fuente"] = data_analisis["fuente"].astype(str).str.strip().str.lower()
data_analisis["distrito"] = data_analisis["distrito_oficial"].astype(str).str.strip().str.title()
data_analisis["inmueble"] = data_analisis["inmueble"].astype(str).str.strip().str.lower()
data_analisis["operacion"] = data_analisis["operacion"].astype(str).str.strip().str.lower()
data_analisis["precio_pen"] = data_analisis["soles"]
data_analisis["precio_usd"] = data_analisis["dolares"]

## solo registros con precios
data_analisis["sin_precio"] = (data_analisis["precio_pen"].isna() & data_analisis["precio_usd"].isna()).astype(int)
data_analisis = data_analisis.query("sin_precio == 0").copy()

# AREA robusta
tmp = data_analisis["area"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mm = tmp.str.split(" a ", n=1, expand=True)
data_analisis["area_min"] = pd.to_numeric(mm[0].str.replace(",", "."), errors="coerce")
data_analisis["area_max"] = pd.to_numeric(mm[1].str.replace(",", "."), errors="coerce")
data_analisis["area_promedio"] = data_analisis[["area_min","area_max"]].mean(axis=1)

data_analisis

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,enlace,fecha,file_origin,distrito,precio_pen,precio_usd,sin_precio,area_min,area_max,area_promedio
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,2300.0,683.0,0,61.0,NaN,61.0
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,1700.0,NaN,0,60.0,NaN,60.0
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,2500.0,NaN,0,75.0,NaN,75.0
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,1550.0,NaN,0,32.0,NaN,32.0
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23,data_alquiler_departamentos_adondevivir_Lima C...,Lima Cercado,1500.0,NaN,0,50.0,NaN,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49846,urbania,departamentos,venta,Barranco,Lima Top,"Pasaje Bresciani 1\nBarranco, Lima","S/ 6,146,200 · USD 1,580,000",6146200.0,1580000.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv,Barranco,6146200.0,1580000.0,0,638.0,NaN,638.0
49847,urbania,departamentos,venta,Barranco,Lima Top,"Jirón Tumay 095\nBarranco, Lima","S/ 603,000 · USD 167,500",603000.0,167500.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv,Barranco,603000.0,167500.0,0,62.0,NaN,62.0
49848,urbania,departamentos,venta,Barranco,Lima Top,"martinto 150\nBarranco, Lima","S/ 562,800 · USD 168,000",562800.0,168000.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv,Barranco,562800.0,168000.0,0,86.0,NaN,86.0
49849,urbania,departamentos,venta,Barranco,Lima Top,"Ignacio mariategui\nBarranco, Lima","S/ 351,750 · USD 105,000",351750.0,105000.0,NaN,...,https://urbania.pe/inmueble/clasificado/veclap...,2025-12-22,data_venta_departamentos_urbania_Lima Top.csv,Barranco,351750.0,105000.0,0,76.0,NaN,76.0


In [18]:
# DORMITORIO robusto
tmpd = data_analisis["dormitorio"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mmd = tmpd.str.split(" a ", n=1, expand=True)
data_analisis["domitorio_min"] = pd.to_numeric(mmd[0], errors="coerce")
data_analisis["domitorio_max"] = pd.to_numeric(mmd[1], errors="coerce")

# Precio por m2 (división segura)
den = data_analisis["area_promedio"].replace(0, np.nan)
data_analisis["precio_m2_pen"] = (data_analisis["precio_pen"] / den).round(2)

# Flags
data_analisis["tiene_mantenimiento"] = data_analisis["mantenimiento"].notna().astype(int)
data_analisis["tiene_estacionamientos"] = data_analisis["estacionamientos"].notna().astype(int)
data_analisis["banios"] = data_analisis["caracteristica"].str.extract(r"(\d+)\s*bañ").astype("Int64")
data_analisis.head()

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,sin_precio,area_min,area_max,area_promedio,domitorio_min,domitorio_max,precio_m2_pen,tiene_mantenimiento,tiene_estacionamientos,banios
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,0,61.0,NaN,61.0,2.0,NaN,37.70,0,0,2
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,0,60.0,NaN,60.0,3.0,NaN,28.33,1,0,2
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,0,75.0,NaN,75.0,3.0,NaN,33.33,0,0,2
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,0,32.0,NaN,32.0,1.0,NaN,48.44,1,0,1
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,0,50.0,NaN,50.0,2.0,NaN,30.00,1,0,2


In [19]:
data_analisis['distrito_categoria'] = data_analisis['zona'].astype('category')

In [20]:
data_analisis.head()

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,area_min,area_max,area_promedio,domitorio_min,domitorio_max,precio_m2_pen,tiene_mantenimiento,tiene_estacionamientos,banios,distrito_categoria
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,61.0,NaN,61.0,2.0,NaN,37.70,0,0,2,Lima Centro
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,60.0,NaN,60.0,3.0,NaN,28.33,1,0,2,Lima Centro
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,75.0,NaN,75.0,3.0,NaN,33.33,0,0,2,Lima Centro
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,32.0,NaN,32.0,1.0,NaN,48.44,1,0,1,Lima Centro
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,50.0,NaN,50.0,2.0,NaN,30.00,1,0,2,Lima Centro


In [21]:
# Se crean columnas categóricas para los rangos de precio y área, que se usarán en los filtros de las pestañas.
data_analisis['precio_pen'] = pd.to_numeric(data_analisis['precio_pen'], errors='coerce')
data_analisis['precio_usd'] = pd.to_numeric(data_analisis['precio_usd'], errors='coerce')
data_analisis['area'] = pd.to_numeric(data_analisis['area'], errors='coerce')

In [22]:
data_analisis[data_analisis["operacion"]=="alquiler"]["precio_pen"].describe()

count    7.617000e+03
mean     8.203685e+03
std      8.032473e+04
min      4.000000e+02
25%      2.100000e+03
50%      2.800000e+03
75%      4.358000e+03
max      2.500000e+06
Name: precio_pen, dtype: float64

In [23]:
data_analisis[data_analisis["operacion"]=="venta"]["precio_pen"].describe()

count    4.206700e+04
mean     2.059305e+06
std      2.557716e+07
min      1.000000e+00
25%      4.994000e+05
50%      8.510000e+05
75%      1.613365e+06
max      1.685000e+09
Name: precio_pen, dtype: float64

In [24]:
# Bins y labels para precio de venta (Soles)
bins_alquiler = [-1, 1500, 2500, 3500, 4500, float('inf')]
labels_alquiler = ["Hasta S/1.5k", "De S/1.5k a S/2.5k", "De S/2.5 a S/3.5k", "De S/3.5k a S/4.5k", "De S/4.5k a más"]
data_analisis['precio_alquiler_agp'] = pd.cut(data_analisis[data_analisis["operacion"]=="alquiler"]["precio_pen"], bins=bins_alquiler, labels=labels_alquiler, right=False)

# Bins y labels para precio de venta (Dólares)
bins_venta = [-1, 50000, 100000, 200000, 500000, float('inf')]
labels_venta = ["Hasta $ 50k", "De $ 50k a $ 100k", "De $ 100k a $ 200k", "De $ 200k a $ 500k", "De $ 500k a más"]
data_analisis['precio_venta_agp'] = pd.cut(data_analisis[data_analisis["operacion"]=="venta"]['precio_usd'], bins=bins_venta, labels=labels_venta, right=False)

# Bins y labels para área (m²)
bins_area = [-1, 50, 100, 200, 300, float('inf')]
labels_area = ["Hasta 50m2", "De 50m2 a 100m2", "De 100m2 a 200m2", "De 200m2 a 300m2", "De 300m2 a más"]
data_analisis['area_agp'] = pd.cut(data_analisis['area'], bins=bins_area, labels=labels_area, right=False)

## Estacionamiento
data_analisis["estacionamiento_gp"] = data_analisis["estacionamientos"].apply(lambda x: "Si" if x > 0 else "No")

In [22]:
data_analisis["distrito_oficial"].value_counts()

distrito_oficial
Miraflores                 8887
Santiago de Surco          8864
San Isidro                 5636
La Molina                  3505
San Borja                  2529
San Miguel                 2274
Barranco                   2030
Surquillo                  1501
Chorrillos                 1464
Jesús María                1171
Magdalena                  1120
Lima Cercado               1076
Pueblo Libre                960
Lince                       851
Ate Vitarte                 719
Los Olivos                  579
San Martin de Porres        534
Punta Hermosa               525
La Victoria                 487
San Juan de Lurigancho      472
San Bartolo                 404
Comas                       386
San Juan de Miraflores      359
Carabayllo                  271
Lurín                       254
Breña                       253
Rimac                       190
Independencia               172
Santa Maria del Mar         155
Villa El Salvador           154
Pachacamac             

In [23]:
data_analisis.columns

Index(['fuente', 'inmueble', 'operacion', 'distrito_oficial', 'zona',
       'direccion', 'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin', 'distrito', 'precio_pen',
       'precio_usd', 'sin_precio', 'area_min', 'area_max', 'area_promedio',
       'domitorio_min', 'domitorio_max', 'precio_m2_pen',
       'tiene_mantenimiento', 'tiene_estacionamientos', 'banios',
       'distrito_categoria', 'precio_alquiler_agp', 'precio_venta_agp',
       'area_agp', 'estacionamiento_gp'],
      dtype='object')

In [24]:
data_analisis.head()

,fuente,inmueble,operacion,distrito_oficial,zona,direccion,precio,soles,dolares,variacion,...,domitorio_max,precio_m2_pen,tiene_mantenimiento,tiene_estacionamientos,banios,distrito_categoria,precio_alquiler_agp,precio_venta_agp,area_agp,estacionamiento_gp
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,NaN,...,NaN,37.70,0,0,2,Lima Centro,De S/1.5k a S/2.5k,NaN,De 50m2 a 100m2,No
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,5%,...,NaN,28.33,1,0,2,Lima Centro,De S/1.5k a S/2.5k,NaN,De 50m2 a 100m2,No
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,NaN,...,NaN,33.33,0,0,2,Lima Centro,De S/2.5 a S/3.5k,NaN,De 50m2 a 100m2,No
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,NaN,...,NaN,48.44,1,0,1,Lima Centro,De S/1.5k a S/2.5k,NaN,Hasta 50m2,No
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,NaN,...,NaN,30.00,1,0,2,Lima Centro,De S/1.5k a S/2.5k,NaN,De 50m2 a 100m2,No


In [25]:
# Orden final
cols_final = [
    'fuente','inmueble','operacion', 'distrito', 'zona','direccion',
    'precio','soles','dolares','precio_pen','precio_usd','variacion',
    'mantenimiento','tiene_mantenimiento','caracteristica',
    'area','area_min','area_max','area_promedio', 'area_agp', 'precio_m2_pen', 'precio_venta_agp', 'precio_alquiler_agp',
    'dormitorio','domitorio_min','domitorio_max','banios', 'estacionamientos','tiene_estacionamientos', 'estacionamiento_gp',
    'detalle','enlace','fecha'
]
data_analisis = data_analisis[cols_final].copy()

In [26]:
data_analisis.head(100)

,fuente,inmueble,operacion,distrito,zona,direccion,precio,soles,dolares,precio_pen,...,dormitorio,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,2300.0,...,2,2.0,NaN,2,NaN,0,No,Departamento de estreno con Balcón y Excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,1700.0,...,3,3.0,NaN,2,NaN,0,No,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,2500.0,...,3,3.0,NaN,2,NaN,0,No,Contacto: malena peláez // cel: 993*306*450. S...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,1550.0,...,1,1.0,NaN,1,NaN,0,No,Alquiler de Departamento en Lima Cercado - Urb...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,1500.0,...,2,2.0,NaN,2,NaN,0,No,"Con un área de 50m2, Ubicado en el Piso 14 con...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,Avenida Paseo de la Republica 2199\nLa Victori...,"S/ 1,900",1900.0,NaN,1900.0,...,1,1.0,NaN,1,NaN,0,No,Se alquila moderno loft de 45 m² en el piso 27...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
97,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"ON APARTMENTS\nLa Victoria, Lima","S/ 2,400 · USD 705",2400.0,705.0,2400.0,...,3,3.0,NaN,2,1.0,1,Si,Alquilo moderno departamento en Santa Catalina...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
98,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"Javier Prado 1071\nCorpac, La Victoria","S/ 2,200",2200.0,NaN,2200.0,...,1,1.0,NaN,1,NaN,0,No,Súper Céntrico ubicación inmejorable departame...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
99,adondevivir,departamentos,alquiler,La Victoria,Lima Centro,"Audacity\nSanta Catalina, La Victoria","S/ 1,750",1750.0,NaN,1750.0,...,NaN,NaN,NaN,1,NaN,0,No,"Alquilo Exclusivo Loft en Javier Prado! S/ 1, ...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23


In [36]:
data_analisis[data_analisis["dormitorio"]==20]

,fuente,inmueble,operacion,distrito,zona,direccion,precio,soles,dolares,precio_pen,...,dormitorio,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha
8053,adondevivir,casas,venta,Santa Anita,Lima Este,"avenida los cipreses\nLos Ficus, Santa Anita","S/ 1,561,500 · USD 450,000",1561500.0,450000.0,1561500.0,...,20.0,20.0,NaN,8,NaN,0,No,¡Gran oportunidad de inversión en una de las z...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8144,adondevivir,casas,venta,San Juan De Lurigancho,Lima Este,"Coyllur\nZarate, San Juan de Lurigancho","S/ 965,000 · USD 287,000",965000.0,287000.0,965000.0,...,20.0,20.0,NaN,8,1.0,1,Si,Quieres que tu casa te genere activos? Distrib...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8718,adondevivir,casas,venta,San Martin De Porres,Lima Norte,"FILADELFIA 2882\nSan Martín de Porres, Lima","S/ 893,000 · USD 265,000",893000.0,265000.0,893000.0,...,20.0,20.0,NaN,10,1.0,1,Si,Ideal para inversión – av. Perú con av. Univer...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8726,adondevivir,casas,venta,San Martin De Porres,Lima Norte,"Calle Eduardo Girardo N° 191 -193, SAN Martín ...","S/ 1,645,000 · USD 470,000",1645000.0,470000.0,1645000.0,...,20.0,20.0,NaN,8,1.0,1,Si,Casa multifamiliar en venta ubicada en San Mar...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8868,adondevivir,casas,venta,San Martin De Porres,Lima Norte,"FILADELFIA 2882\nSan Martín de Porres, Lima","S/ 893,000 · USD 265,000",893000.0,265000.0,893000.0,...,20.0,20.0,NaN,10,1.0,1,Si,Ideal para inversión – av. Perú con av. Univer...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8869,adondevivir,casas,venta,San Martin De Porres,Lima Norte,"Calle Eduardo Girardo N° 191 -193, SAN Martín ...","S/ 1,645,000 · USD 470,000",1645000.0,470000.0,1645000.0,...,20.0,20.0,NaN,8,1.0,1,Si,Casa multifamiliar en venta ubicada en San Mar...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
8965,adondevivir,casas,venta,San Martin De Porres,Lima Norte,urbanizacion virgen de fatima\nSan Martín de P...,"S/ 1,120,000 · USD 320,000",1120000.0,320000.0,1120000.0,...,20.0,20.0,NaN,11,NaN,0,No,Excelente oportunidad de inversión! Venta de c...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
9218,adondevivir,casas,venta,Chorrillos,Lima Sur,"Qxvj+x5c, Chorrillos 15067, Perú\nChorrillos, ...","S/ 1,260,000 · USD 360,000",1260000.0,360000.0,1260000.0,...,20.0,20.0,NaN,19,6.0,1,Si,Inversión rentable en chorrillos¡Oportunidad d...,https://www.adondevivir.com/propiedades/clasif...,2025-12-24
13197,urbania,casas,venta,Santa Anita,Lima Este,"avenida los cipreses\nLos Ficus, Santa Anita","S/ 1,561,500 · USD 450,000",1561500.0,450000.0,1561500.0,...,20.0,20.0,NaN,8,NaN,0,No,¡Gran oportunidad de inversión en una de las z...,https://urbania.pe/inmueble/clasificado/veclca...,2025-12-24
13250,urbania,casas,venta,San Juan De Lurigancho,Lima Este,"Coyllur\nZarate, San Juan de Lurigancho","S/ 965,000 · USD 287,000",965000.0,287000.0,965000.0,...,20.0,20.0,NaN,8,1.0,1,Si,Quieres que tu casa te genere activos? Distrib...,https://urbania.pe/inmueble/clasificado/veclca...,2025-12-24


In [27]:
zonas_lima = {
        'Lima Top': ['Miraflores', 'San Isidro', 'La Molina', 'Santiago De Surco', 'San Borja', 'Barranco'],
        'Lima Moderna': ['Jesús María', 'Lince', 'Magdalena', 'San Miguel', 'Pueblo Libre', 'Surquillo'],
        'Lima Centro': ['Lima Cercado', 'Breña', 'La Victoria', 'Rimac', 'San Luis'],
        'Lima Este': ['Ate Vitarte', 'Cieneguilla', 'Chaclacayo', 'Chosica Lurigancho', 'Santa Anita', 'El Agustino', 'San Juan De Lurigancho'],
        'Lima Norte': ['Carabayllo', 'Comas', 'Independencia', 'Los Olivos', 'Puente Piedra', 'San Martin De Porres', 'Ancón', 'Santa Rosa'],
        'Lima Sur': ['Chorrillos', 'Lurín', 'Pachacamac', 'San Juan De Miraflores', 'Villa El Salvador', 'Villa Maria Del Triunfo', 'Pucusana', 'Punta Hermosa', 'Punta Negra', 'San Bartolo', 'Santa Maria Del Mar']
    }

distrito_a_zona = {distrito: zona for zona, distritos_en_zona in zonas_lima.items() for distrito in distritos_en_zona}

In [28]:
data_analisis["zona"] = data_analisis["distrito"].map(distrito_a_zona).fillna('Otra Zona')

In [29]:
data_analisis[data_analisis["zona"]=="Otra Zona"]["distrito"].unique()

array([], dtype=object)

In [37]:
data_analisis.columns

Index(['fuente', 'inmueble', 'operacion', 'distrito', 'zona', 'direccion',
       'precio', 'soles', 'dolares', 'precio_pen', 'precio_usd', 'variacion',
       'mantenimiento', 'tiene_mantenimiento', 'caracteristica', 'area',
       'area_min', 'area_max', 'area_promedio', 'area_agp', 'precio_m2_pen',
       'precio_venta_agp', 'precio_alquiler_agp', 'dormitorio',
       'domitorio_min', 'domitorio_max', 'banios', 'estacionamientos',
       'tiene_estacionamientos', 'estacionamiento_gp', 'detalle', 'enlace',
       'fecha'],
      dtype='object')

In [38]:
data_analisis["dormitorio"].describe()

count     48472
unique       61
top           3
freq      19284
Name: dormitorio, dtype: object

In [39]:
data_analisis["domitorio_min"].describe()

count    48472.000000
mean         3.030678
std          1.713840
min          1.000000
25%          2.000000
50%          3.000000
75%          3.000000
max         55.000000
Name: domitorio_min, dtype: float64

In [41]:
data_analisis["dormitorios"] = pd.cut(data_analisis['domitorio_min'],
                                    bins=[1, 2, 3, 4, 5, float('inf')],
                                    labels=['1 Dormitorio', '2 Dormitorios', '3 Dormitorios', '4 Dormitorios', '5 o más Dormitorios'],
                                    right=False)

In [45]:
data_analisis[["dormitorio", "domitorio_min", "dormitorios", "detalle"]].head(20)

,dormitorio,domitorio_min,dormitorios,detalle
0,2,2.0,2 Dormitorios,Departamento de estreno con Balcón y Excelente...
1,3,3.0,3 Dormitorios,Se Alquila Departamento bien iluminado con Bal...
2,3,3.0,3 Dormitorios,Contacto: malena peláez // cel: 993*306*450. S...
3,1,1.0,1 Dormitorio,Alquiler de Departamento en Lima Cercado - Urb...
4,2,2.0,2 Dormitorios,"Con un área de 50m2, Ubicado en el Piso 14 con..."
5,3,3.0,3 Dormitorios,Alquilo departamento de estreno en 2do piso en...
6,3,3.0,3 Dormitorios,Alquilo moderno departamento en lima cercado. ...
7,3,3.0,3 Dormitorios,Alquiler de Departamento de Estreno en Condomi...
8,3,3.0,3 Dormitorios,Departamento de 73m2 muy bien distribuidos. Sa...
9,3,3.0,3 Dormitorios,Departamento de Estreno en Alquiler – 3 Dormit...


In [43]:
data_analisis["dormitorios"].value_counts()

dormitorios
3 Dormitorios          21411
2 Dormitorios          10887
4 Dormitorios           6531
1 Dormitorio            5140
5 o más Dormitorios     4503
Name: count, dtype: int64

In [44]:
print("\nData Final creada satisfacotiramente!")

# Export
out_path = DATA_OUT / "data_dondevivir_analisis.csv"
data_analisis.to_csv(out_path, sep="|", index=False)
print("\nData Final creada satisfactoriamente ->", out_path)


Data Final creada satisfacotiramente!

Data Final creada satisfactoriamente -> C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed\data_dondevivir_analisis.csv


In [32]:
data_analisis[data_analisis["distrito"]=="Independencia"]["zona"].unique()

array(['Lima Norte'], dtype=object)

In [34]:
path_final_ = rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed\data_dondevivir_analisis.csv"
data_final = pd.read_csv(path_final_, sep="|") 

data_final.head()

C:\Users\PC\AppData\Local\Temp\ipykernel_25784\129289585.py:2: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data_final = pd.read_csv(path_final_, sep="|")


,fuente,inmueble,operacion,distrito,zona,direccion,precio,soles,dolares,precio_pen,...,dormitorio,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Mariano Carranza al 400\nSanta Beatriz, Lima C...","S/ 2,300 · USD 683",2300.0,683.0,2300.0,...,2,2.0,NaN,2.0,NaN,0,No,Departamento de estreno con Balcón y Excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,700\n5%",1700.0,NaN,1700.0,...,3,3.0,NaN,2.0,NaN,0,No,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Parque de la Reserva 736\nLima Cercado, Lima","S/ 2,500",2500.0,NaN,2500.0,...,3,3.0,NaN,2.0,NaN,0,No,Contacto: malena peláez // cel: 993*306*450. S...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Alejandro Bertello Cdra\nPando, Lima Cercado","S/ 1,550",1550.0,NaN,1550.0,...,1,1.0,NaN,1.0,NaN,0,No,Alquiler de Departamento en Lima Cercado - Urb...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,"Av. Oscar R. Benavides\nLima Cercado, Lima","S/ 1,500",1500.0,NaN,1500.0,...,2,2.0,NaN,2.0,NaN,0,No,"Con un área de 50m2, Ubicado en el Piso 14 con...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23


pr analizar!

## Reasignacion de Venta o Alquiler, segun el precio

In [124]:
data_analisis[(data_analisis["distrito"]=="Magdalena")&(data_analisis["operacion"] == "alquiler")]

,fuente,inmueble,operacion,distrito,zona,direccion,precio,soles,dolares,precio_pen,...,dormitorio,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha
577,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Je. Tacna 300 Magdalena\nMagdalena, Lima","S/ 2,700 · USD 800",2700.0,800.0,2700.0,...,2.0,2.0,NaN,2,NaN,0,No,"Se alquila moderno departamento de 60 m2, con ...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23
578,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Francisco Graña 600\nMagdalena, Lima","S/ 3,300",3300.0,NaN,3300.0,...,3.0,3.0,NaN,2,1.0,1,Si,¡Vive donde la modernidad se encuentra con el ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
579,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Av Javier Prado Oeste\nMagdalena, Lima","S/ 3,044 · USD 900",3044.0,900.0,3044.0,...,2.0,2.0,NaN,2,1.0,1,Si,De estreno Departamento de 65m2. Con 02 dormit...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
580,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Av Brasil,cuadra 30, Magdalena, Lima\nMagdalen...","S/ 2,100\n8%",2100.0,NaN,2100.0,...,1.0,1.0,NaN,2,1.0,1,Si,"Iluminado departamento en alquiler, 50 m2, cer...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23
581,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Avenida Salaverry 2900\nMagdalena, Lima","S/ 3,360 · USD 1,000",3360.0,1000.0,3360.0,...,2.0,2.0,NaN,2,1.0,1,Si,Alquiler de acogedor departamento ubicado entr...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"AV EJERCITO 224\nUdima, Magdalena","S/ 1,800 · USD 529",1800.0,529.0,1800.0,...,2.0,2.0,NaN,2,NaN,0,No,55 m² tot. 2 baños. 2 dorm. Estreno en piso 3....,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
685,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Junin 881 Magdalena\nMagdalena, Lima","S/ 2,150",2150.0,NaN,2150.0,...,3.0,3.0,NaN,2,NaN,0,No,Se alquila departamento de 3 dormitorios ubica...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
686,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"Jirón Larco Herrera 712\nOyague, Magdalena","S/ 2,000",2000.0,NaN,2000.0,...,1.0,1.0,NaN,1,1.0,1,Si,Se alquila departamento de 77 m² con excelente...,https://www.adondevivir.com/propiedades/clasif...,2025-12-23
687,adondevivir,departamentos,alquiler,Magdalena,Lima Moderna,"AV DEL EJERCITO 231\nMagdalena, Lima","S/ 3,000",3000.0,NaN,3000.0,...,2.0,2.0,NaN,3,1.0,1,Si,"Departamento en Alquiler – 99 m² en Magdalena,...",https://www.adondevivir.com/propiedades/clasif...,2025-12-23


In [134]:
stats_alquiler = data_analisis[data_analisis["operacion"]=="alquiler"].groupby(['distrito']).agg(
    conteo   = ('precio_pen', 'count'),
    promedio = ('precio_pen', 'mean'),
    mediana  = ('precio_pen', 'median'),
    minimo   = ('precio_pen', 'min'),
    maximo   = ('precio_pen', 'max'),
    p80      = ('precio_pen', lambda x: x.quantile(0.8))
)

stats_alquiler

,conteo,promedio,mediana,minimo,maximo,p80
distrito,,,,,,
Ancón,4,31320.000000,31320.0,31320.0,31320.0,31320.0
Ate Vitarte,114,1654.964912,1580.0,550.0,3200.0,2260.0
Barranco,643,8965.138414,2550.0,1300.0,1541000.0,3960.0
Breña,31,1848.387097,1800.0,1000.0,3000.0,2100.0
Carabayllo,52,816.346154,750.0,500.0,1600.0,938.0
Chaclacayo,11,1323.636364,1500.0,720.0,1900.0,1900.0
Chorrillos,182,2045.851648,1800.0,800.0,9000.0,2200.0
Chosica Lurigancho,8,875.000000,900.0,800.0,900.0,900.0
Comas,125,1088.480000,990.0,600.0,2500.0,1250.0


In [135]:
data_fix = data_analisis.copy()

In [ ]:
https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-miraflores-lima-145461733?n_src=Listado&n_pills=%C3%81rea+de+lavander%C3%ADa&n_pg=24&n_pos=14

In [ ]:
https://www.adondevivir.com/propiedades/clasificado/alclapin-alquiler-miraflores-av-reducto-148119154.html?n_src=Listado&n_pills=Piscina&n_pg=27&n_pos=21


In [ ]:
https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-miraflores-lima-1-dormitorio-147891827?n_src=Listado&n_pills=Ascensor&n_pg=27&n_pos=20


In [ ]:
https://urbania.pe/inmueble/clasificado/alclapin-alquiler-de-departamento-en-malecon-reserva-miraflores-3-dormitorios-145697528?n_src=Listado&n_pills=Parrilla&n_pg=12&n_pos=8


In [161]:
def cambio_operacion_alquiler(distrito, operacion):
    
    data_distrito = data_fix[(data_fix["distrito"] == distrito) & (data_fix["operacion"]== operacion)] 
    metrica = data_distrito["precio_pen"].median()
    
    data_filtrada = data_distrito[data_distrito["precio_pen"]>=metrica][["operacion", "precio_pen", "precio_usd", "detalle", "enlace"]]
    
    data_filtrada["Flag_Operacion"] = data_filtrada["detalle"].str.contains(r"\b(venta|vendo|se vende|en venta)\b", case = False, na=False).astype(int)
    
    return data_filtrada

cambio_operacion_alquiler("Miraflores", "alquiler")

C:\Users\PC\AppData\Local\Temp\ipykernel_18476\1254414456.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_filtrada["Flag_Operacion"] = data_filtrada["detalle"].str.contains(r"\b(venta|vendo|se vende|en venta)\b", case = False, na=False).astype(int)


,operacion,precio_pen,precio_usd,detalle,enlace,Flag_Operacion
1678,alquiler,6230.0,1780.0,Departamento Amoblado en Alquiler con Balcón y...,https://www.adondevivir.com/propiedades/clasif...,0
1679,alquiler,5130.0,1500.0,Malecón cisneros Cdra 1 Primer Piso. Área 145m...,https://www.adondevivir.com/propiedades/clasif...,0
1680,alquiler,3855.0,1135.0,"Departamento totalmente amoblado y equipado, u...",https://www.adondevivir.com/propiedades/clasif...,0
1681,alquiler,3850.0,1050.0,Departamento con cocina equipada. vive Y conec...,https://www.adondevivir.com/propiedades/clasif...,0
1682,alquiler,4877.0,1450.0,Alquiler en Miraflores – Departamento Full Amo...,https://www.adondevivir.com/propiedades/clasif...,0
...,...,...,...,...,...,...
5742,alquiler,3800.0,NaN,Departamento en pis0 16 con vista al mar. Cons...,https://urbania.pe/inmueble/clasificado/alclap...,0
5744,alquiler,6960.0,2000.0,"Piso 7. Guardianía, no se permiten mascotas, d...",https://urbania.pe/inmueble/clasificado/alclap...,0
5747,alquiler,5000.0,NaN,Alquiler departamento suite exclusivo – mirafl...,https://urbania.pe/inmueble/clasificado/alclap...,0
5755,alquiler,3900.0,NaN,"Alquiler Departamento Amoblado – Av. Arequipa,...",https://urbania.pe/inmueble/clasificado/alclap...,0
